# Food Image Classifcation

### Dataset

We are going learn [food images](https://www.kaggle.com/kmader/food41) from kaggle

### Use the kaggle-cli to download the image

```kaggle datasets download -d kmader/food41```

In [1]:
import os
import torch

In [2]:
CUDA  = torch.cuda.is_available()

In [3]:
HOME = os.environ["HOME"]
HOME

'/home/paperspace'

In [4]:
DATA  = HOME+"/.kaggle/datasets/kmader/food41/"
DATA

'/home/paperspace/.kaggle/datasets/kmader/food41/'

In [20]:
!!du -sh {DATA}*

['5.3G\t/home/paperspace/.kaggle/datasets/kmader/food41/food41.zip',
 '343M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n1000_r384x384x3.h5',
 '18M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x1.h5',
 '30M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x3.h5',
 '68M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x1.h5',
 '115M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x3.h5',
 '25M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x1.h5',
 '44M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x3.h5',
 '1.9M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x1.h5',
 '3.2M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x3.h5',
 '6.7M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r64x64x1.h5',
 '12M\t/home/paperspace/.kaggle/datasets/

In [23]:
%mkdir -p {DATA}images
%mkdir -p {DATA}meta
%ls -l {DATA}

total 11240352
-rw-rw-r-- 1 paperspace paperspace 5681757245 Aug  2 01:45 food41.zip
-rw-rw-r-- 1 paperspace paperspace  359407496 Aug  2 01:45 food_c101_n1000_r384x384x3.h5
-rw-rw-r-- 1 paperspace paperspace   18236331 Aug  2 01:45 food_c101_n10099_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace   30452874 Aug  2 01:45 food_c101_n10099_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace   70546750 Aug  2 01:45 food_c101_n10099_r64x64x1.h5
-rw-rw-r-- 1 paperspace paperspace  120306214 Aug  2 01:45 food_c101_n10099_r64x64x3.h5
-rw-rw-r-- 1 paperspace paperspace   26042428 Aug  2 01:45 food_test_c101_n1000_r128x128x1.h5
-rw-rw-r-- 1 paperspace paperspace   45897229 Aug  2 01:45 food_test_c101_n1000_r128x128x3.h5
-rw-rw-r-- 1 paperspace paperspace    1938115 Aug  2 01:45 food_test_c101_n1000_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace    3294763 Aug  2 01:45 food_test_c101_n1000_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace    6968764 Aug  2 01:45 food_test_c101_n1000_r64x64x1.h5
-rw-rw-r--

Unzip and calculate the line of sub-folders: categories

In [24]:
!!cd {DATA}/images; unzip ../images.zip > unzip.log;rm -rf unzip.log; ls -l|wc -l

['102']

In [29]:
!!cd {DATA}/meta; unzip -q ../meta.zip; ls -l meta

['total 4140',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Jul  9  2014 classes.txt',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Sep 23  2013 labels.txt',
 '-rw-rw-r-- 1 paperspace paperspace  566868 Sep 23  2013 test.json',
 '-rw-rw-r-- 1 paperspace paperspace  489429 Sep 23  2013 test.txt',
 '-rw-rw-r-- 1 paperspace paperspace 1697751 Sep 21  2013 train.json',
 '-rw-rw-r-- 1 paperspace paperspace 1468812 Sep 21  2013 train.txt']

In [5]:
META = DATA + "meta/meta/"
META

'/home/paperspace/.kaggle/datasets/kmader/food41/meta/meta/'

In [6]:
for m_file in os.listdir(META):
    print(m_file, "preview","="*20)
    print(open(META+m_file).read()[:50])

train.json preview ====================
{"churros": ["churros/1004234", "churros/1013460",
classes.txt preview ====================
apple_pie
baby_back_ribs
baklava
beef_carpaccio
be
train.txt preview ====================
apple_pie/1005649
apple_pie/1014775
apple_pie/1026
test.json preview ====================
{"churros": ["churros/1061830", "churros/1064042",
test.txt preview ====================
apple_pie/1011328
apple_pie/101251
apple_pie/10343
labels.txt preview ====================
Apple pie
Baby back ribs
Baklava
Beef carpaccio
Be


### Datasets

In [6]:
IMG = DATA+"images/"
IMG

'/home/paperspace/.kaggle/datasets/kmader/food41/images/'

In [7]:
from torch.utils.data import DataLoader
from torch import nn
from ray.matchbox import Trainer
from torchvision.models.densenet import densenet121 as feature_extractor
from torch.nn import functional as F
from torch.optim import Adam
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [8]:
SCALE = 224
transform = transforms.Compose([
    transforms.Resize((SCALE,SCALE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
])

In [9]:
img_set = ImageFolder(IMG,transform = transform, )

#### Train /Valid Split

In [10]:
import numpy as np

In [11]:
trn_set = ImageFolder(IMG,transform = transform, )
val_set = ImageFolder(IMG,transform = transform, )

In [12]:
val_pick = np.random.rand(len(img_set.samples))>0.8
trn_pick = ~val_pick

In [13]:
trn_set.samples = np.array(img_set.samples)[trn_pick].tolist()
val_set.samples = np.array(img_set.samples)[val_pick].tolist()

In [14]:
len(trn_set),len(val_set)

(80696, 20304)

#### Test dataset

In [15]:
gen = iter(DataLoader(trn_set,batch_size=2,shuffle=True))
next(gen)

[tensor([[[[ 1.4783,  1.4612,  1.4098,  ...,  1.3755,  1.4269,  1.4269],
           [ 1.4269,  1.3927,  1.3755,  ...,  1.4612,  1.4440,  1.4612],
           [ 1.4440,  1.4269,  1.4440,  ...,  1.4098,  1.3755,  1.3755],
           ...,
           [ 1.3413,  1.3413,  1.2899,  ...,  1.1187,  1.0673,  0.9817],
           [ 1.2728,  1.3413,  1.3927,  ...,  1.0331,  0.8789,  0.8961],
           [ 1.4783,  1.4098,  1.2557,  ...,  0.8618,  0.9817,  0.9988]],
 
          [[ 1.4132,  1.3957,  1.3431,  ...,  1.1681,  1.2031,  1.1856],
           [ 1.3606,  1.3256,  1.3081,  ...,  1.2556,  1.2031,  1.2381],
           [ 1.3782,  1.3606,  1.3782,  ...,  1.2031,  1.1681,  1.1506],
           ...,
           [ 1.2206,  1.2206,  1.1681,  ...,  0.8880,  0.8354,  0.7479],
           [ 1.1506,  1.2206,  1.2731,  ...,  0.8004,  0.6429,  0.6604],
           [ 1.3606,  1.2906,  1.1331,  ...,  0.6254,  0.7479,  0.7654]],
 
          [[ 1.4722,  1.4548,  1.4025,  ...,  1.0888,  1.1237,  1.1237],
           [ 

In [16]:
def conv_layer(in_, out_, ks=3, activation = nn.LeakyReLU()):
    return nn.Sequential(*[
        nn.Conv2d(in_, out_, kernel_size=ks, padding=ks//3, bias = False),
        nn.BatchNorm2d(out_,),
        activation,
    ])

def conv_block(in_,out_,nb_layers):
    layers = []
    for i in range(nb_layers):
        if i == 0: layers.append(conv_layer(in_,out_))
        elif i == nb_layers - 1:layers.append(conv_layer(out_,out_,activation = nn.MaxPool2d((2,2))))
        else: layers.append(conv_layer(out_,out_))
        
    return nn.Sequential(*layers)

In [17]:
class Flatten(nn.Module):
    def forward(self, input):
        """
        a pytorch version of Flatten layer
        """
        return input.view(input.size(0), -1)

In [18]:
def argmax(x):
    """
    Arg max of a torch tensor (2 dimensional, dim=1)
    :param x:  torch tensor
    :return: index the of the max
    """
    return torch.max(x, dim=1)[1]

def accuracy(y_pred, y_true):
    """

    :param y_pred: predition of y (will be argmaxed)
    :param y_true: true label of y (index)
    :return:
    """
    return (argmax(y_pred) == y_true).float().mean()

In [19]:
def save_model(model,path):
    """
    model:pytorch model
    path:save to path, end with pkl
    """
    torch.save(model.state_dict(), path)
    
def load_model(model,path):
    model.load_state_dict(torch.load(path))

## Transfer Learning

In [20]:
conv_model = feature_extractor(pretrained=True)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [21]:

dense_conv1 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["conv0","norm0","relu0","pool0","denseblock1","transition1",
                                                                                   "denseblock2","transition2","denseblock3","transition3",]])

dense_conv2 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["denseblock4","norm5"]])

In [22]:
FEATURE_WIDTH = 1024
DENSE_FEATURE = 1024

In [23]:
class  top_half(nn.Module):
    def __init__(self):
        super(top_half,self).__init__()
        self.top_ = nn.Sequential(*[nn.AdaptiveAvgPool2d((1,1)),Flatten(),
                                   nn.Linear(FEATURE_WIDTH,DENSE_FEATURE,bias=False),
                                    nn.BatchNorm1d(DENSE_FEATURE),
                                    nn.LeakyReLU(inplace=True),
                                    nn.Dropout(p=.5),
                                    nn.Linear(DENSE_FEATURE,len(img_set.classes),bias=True),
                                   ])
    def forward(self,x):
        return F.softmax(self.top_(x))

In [24]:
top_half_  = top_half()

if CUDA:
    top_half_.cuda()
    dense_conv1.cuda()
    dense_conv2.cuda()

In [25]:
def action(*args,**kwargs):
    x,y = args[0]
    y = torch.LongTensor(np.array(y).astype(int))
    if CUDA:
        x,y = x.cuda(),y.cuda()
    opt.zero_grad()
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    
    loss.backward()
    opt.step()
    
    return {"loss":loss.item(),
            "acc":acc.item()}

def val_action(*args,**kwargs):
    x,y = args[0]
    y = torch.LongTensor(np.array(y).astype(int))
    if CUDA:
        x,y = x.cuda(),y.cuda()
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    
    return {"loss":loss.item(),
            "acc":acc.item()}

In [26]:
loss_func = nn.CrossEntropyLoss()
opt = Adam(list(dense_conv2.parameters())+list(top_half_.parameters()))

trainer = Trainer(trn_set, val_dataset = val_set, batch_size = 48, print_on = 5)

trainer.action = action
trainer.val_action = val_action

In [27]:
load_model(dense_conv2,"food_dense_conv2.0.0.1.npy")
load_model(top_half_,"food_top.0.0.1.npy")

In [28]:
trainer.train(1)

  0%|          | 0/1682 [00:00<?, ?it/s]/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
⭐[ep_0_i_1039]	acc	0.546✨	loss	4.091:  62%|██████▏   | 1041/1682 [25:23<15:38,  1.46s/it]

KeyboardInterrupt: 

In [29]:
save_model(dense_conv2,"food_dense_conv2.0.0.1.npy")
save_model(top_half_,"food_top.0.0.1.npy")